# Quick Start: MAUDE Database Analysis

**Purpose**: Fast introduction to the `maude_db` library  
**Target Audience**: New users, workshop attendees, Binder demos  
**Data**: 2005 device table only (~10MB download)  
**Runtime**: ~1 minute setup + queries

## What is MAUDE?

MAUDE (Manufacturer and User Facility Device Experience) is the FDA's database of medical device adverse event reports. This library makes it easy to download, query, and analyze this data locally.

## What You'll Learn

- Create a local MAUDE database
- Download FDA data for a single year
- Query devices using SQL
- View and export results

## Important Note

**This is a simplified demo** using device table only (no master table) for Binder speed. For full helper methods with event types, dates, and trends, see **notebook 02**.

## Next Steps

After completing this notebook:
- **Local use**: See `02_getting_started.ipynb` for full setup with helper methods
- **Documentation**: Check `../docs/` for comprehensive guides
- **Advanced examples**: Explore notebooks 03-08 for research workflows

## 1. Setup

Import the library and configure paths. This notebook uses relative paths that work both locally and on Binder.

In [1]:
import sys
from pathlib import Path

# Add src to path for imports
sys.path.insert(0, str(Path().resolve().parent / 'src'))

from maude_db import MaudeDatabase
import pandas as pd

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

print("✓ Imports successful!")

✓ Imports successful!


## 2. Create Database & Download Data

We'll create a local SQLite database and download 2005 device data. This is a small dataset (~10MB) that downloads quickly and is perfect for learning.

**Why 2005?** It's the smallest recent year with good device data coverage, making it ideal for fast demos.

In [2]:
# Configuration
DB_PATH = 'quickstart_demo.db'
DATA_DIR = './maude_data'
YEAR = 2005  # Small dataset, perfect for quick demos

# Create database connection
db = MaudeDatabase(DB_PATH, verbose=True)

print(f"\nDatabase created at: {DB_PATH}")


Database created at: quickstart_demo.db


In [3]:
# Download 2005 data (small and fast!)
print(f"Downloading {YEAR} data from FDA...\n")
print("Note: Using 2005 data for speed. For recent data, see other notebooks.\n")

db.add_years(
    years=YEAR,
    tables=['device'],  # Just device table for speed (~10MB)
    download=True,
    data_dir=DATA_DIR
)

print("\n✓ Download complete!")
print("\nNote: This demo uses device data only for speed.")
print("For full analysis with event types and dates, see notebook 02.")


Note: Using 2005 data for speed. For recent data, see other notebooks.


Grouping years by file for optimization...

  Using cached device2005.zip

Processing data files...

device for year 2005 already loaded and unchanged, skipping

Creating indexes...

Database update complete

✓ Download complete!

Note: This demo uses device data only for speed.
For full analysis with event types and dates, see notebook 02.


## 3. Explore the Database

Let's see what we have in the database using the `info()` method.

In [4]:
# Show database summary
db.info()


Database: quickstart_demo.db
_maude_load_metadata 2 records
device          99,769 records
text            108,667 records
Database size: 0.06 GB


## 4. Query Devices

Let's search for adverse events related to a specific device type. We'll use "catheter" as an example.

**Note**: This demo uses direct SQL queries on the device table for speed. Other notebooks show the full `query_device()` helper method that joins with the master table.

In [5]:
# Query for catheter devices using SQL
device_name = 'catheter'

print(f"Searching for '{device_name}' devices in {YEAR}...\n")

# Direct SQL query on device table
# Searches both generic name and brand name columns
sql = """
    SELECT *
    FROM device
    WHERE GENERIC_NAME LIKE :device
       OR BRAND_NAME LIKE :device
"""

results = db.query(sql, params={'device': f'%{device_name}%'})

print(f"Found {len(results):,} adverse event reports")
print(f"\nFirst few results:")
results.head()

Searching for 'catheter' devices in 2005...

Found 3,837 adverse event reports

First few results:


,MDR_REPORT_KEY,DEVICE_EVENT_KEY,IMPLANT_FLAG,DATE_REMOVED_FLAG,DEVICE_SEQUENCE_NO,DATE_RECEIVED,BRAND_NAME,GENERIC_NAME,MANUFACTURER_D_NAME,MANUFACTURER_D_ADDRESS_1,MANUFACTURER_D_ADDRESS_2,MANUFACTURER_D_CITY,MANUFACTURER_D_STATE_CODE,MANUFACTURER_D_ZIP_CODE,MANUFACTURER_D_ZIP_CODE_EXT,MANUFACTURER_D_COUNTRY_CODE,MANUFACTURER_D_POSTAL_CODE,DEVICE_OPERATOR,EXPIRATION_DATE_OF_DEVICE,MODEL_NUMBER,CATALOG_NUMBER,LOT_NUMBER,OTHER_ID_NUMBER,DEVICE_AVAILABILITY,DATE_RETURNED_TO_MANUFACTURER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,COMBINATION_PRODUCT_FLAG,UDI-DI,UDI-PUBLIC
0,563000,None,None,None,1,2005-01-03 00:00:00,HYPERGLIDO 4X10 MM,OCCLUSION BALLOON CATHETER,"MICRO THERAPEUTICS, INC.",2 GOODYEAR,None,IRVINE,CA,92618,None,US,92618,0HP,2005-07-01 00:00:00,104-4113,*,79375,*,R,2004-12-07 00:00:00,MJN,None,Y,None,None,None
1,563584,None,None,None,1,2005-01-04 00:00:00,PTCA CATHETERS,MAVERICK2 MONORAIL PTCA CATHETER,BOSTON SCIENTIFIC,BALLYBRIT BUSINESS PARK,None,GALWAY,None,None,None,EI,None,0HP,2007-08-12 00:00:00,MAVERICK2-MR 20/2.5 MM,H7493892820250,6851738,None,R,2005-01-12 00:00:00,LOX,None,Y,None,None,None
2,563585,None,None,None,1,2005-01-04 00:00:00,PTCA CATHETERS,NC MONORAIL PTCA CATHETER,BOSTON SCIENTIFIC,BALLYBRIT BUSINESS PARK,None,GALWAY,None,None,None,EI,None,0HP,2007-05-20 00:00:00,NC MONORAIL 9/2.5,H74921725250,6593619,None,N,None,LOX,None,R,None,None,None
3,563923,None,None,None,1,2005-01-05 00:00:00,PTCA CATHETERS,MAVERICK2 MONORAIL PTCA CATHETER,BOSTON SCIENTIFIC,BALLYBRIT BUSINESS PARK,None,GALWAY,None,None,None,EI,None,0HP,2007-01-01 00:00:00,MAVERICK2-MR 15 / 2.5 MM,H7493892815250,6178628,None,R,2005-01-12 00:00:00,LOX,None,Y,None,None,None
4,563930,None,None,None,1,2005-01-05 00:00:00,PTCA CATHETERS,QUANTUM MAVERICK MONORAIL PTCA CATHETER,BOSTON SCIENTIFIC,ONE-SCIMED PLACE,None,MAPLE GROVE,MN,55311,156,US,55311 1566,0HP,2007-09-21 00:00:00,QUANTUM MAVERICK-MR 12/2.0 MM,H7493808012200,6973125,None,N,None,LOX,None,R,None,None,None


## 5. View Result Details

Let's look at the key columns from the device table in our results.

In [6]:
# Show key columns from device table
key_columns = ['MDR_REPORT_KEY', 'DATE_RECEIVED', 'GENERIC_NAME', 'BRAND_NAME', 'MODEL_NUMBER']
available_cols = [col for col in key_columns if col in results.columns]

print("Key information from the first 10 reports:\n")
results[available_cols].head(10)

Key information from the first 10 reports:



,MDR_REPORT_KEY,DATE_RECEIVED,GENERIC_NAME,BRAND_NAME,MODEL_NUMBER
0,563000,2005-01-03 00:00:00,OCCLUSION BALLOON CATHETER,HYPERGLIDO 4X10 MM,104-4113
1,563584,2005-01-04 00:00:00,MAVERICK2 MONORAIL PTCA CATHETER,PTCA CATHETERS,MAVERICK2-MR 20/2.5 MM
2,563585,2005-01-04 00:00:00,NC MONORAIL PTCA CATHETER,PTCA CATHETERS,NC MONORAIL 9/2.5
3,563923,2005-01-05 00:00:00,MAVERICK2 MONORAIL PTCA CATHETER,PTCA CATHETERS,MAVERICK2-MR 15 / 2.5 MM
4,563930,2005-01-05 00:00:00,QUANTUM MAVERICK MONORAIL PTCA CATHETER,PTCA CATHETERS,QUANTUM MAVERICK-MR 12/2.0 MM
5,563931,2005-01-05 00:00:00,QUANTUM MAVERICK MONORAIL PTCA CATHETER,PTCA CATHETERS,QUANTUM MAVERICK-MR 15 / 3.0 MM
6,563940,2005-01-05 00:00:00,MAVERICK2 MONORAIL PTCA CATHETER,PTCA CATHETERS,MAVERICK2-MR 20/1.5 MM
7,563955,2005-01-05 00:00:00,ULTRA-THIN DIAMOND BALLOON DIL CATHETER,ULTRA-THIN BALLOON DILATATION CATHETER,None
8,563961,2005-01-05 00:00:00,EPIDURAL CATHETER,TUN-L-KATH EPIDURAL CATHETER,155-1520
9,564031,2005-01-04 00:00:00,DIALYSIS CATHETER,ARROW CANNON II DIALYSIS CATHETER,None


## 6. Basic Statistics

Let's get some quick statistics about our query results.

**Note**: We don't have the master table in this demo, so we can't show event types (deaths, injuries, malfunctions) or monthly trends. See notebook 02 for that!

In [7]:
# Count unique device brands
if 'BRAND_NAME' in results.columns:
    unique_brands = results['BRAND_NAME'].nunique()
    print(f"Unique brands: {unique_brands:,}")
    
    print(f"\nTop 10 brands by report count:")
    top_brands = results['BRAND_NAME'].value_counts().head(10)
    print(top_brands)

# Count unique manufacturers
if 'MANUFACTURER_D_NAME' in results.columns:
    unique_mfg = results['MANUFACTURER_D_NAME'].nunique()
    print(f"\nUnique manufacturers: {unique_mfg:,}")

Unique brands: 1,526

Top 10 brands by report count:
BRAND_NAME
MAVERICK2 MONORAIL PTCA CATHETER            268
QUANTUM MAVERICK MONORAIL PTCA CATHETER     158
MAVERICK2 MONORAIL PTCA BALLOON CATHETER     71
INDURA                                       57
RENEGADE                                     57
MAVERICK OTW PTCA CATHETER                   48
INSYTE AUTOGUARD SHIELDED IV CATHETER        42
RENEGADE MICROCATHETER                       39
RENEGADE HI FLO                              37
INTROCAN SAFETY                              36
Name: count, dtype: int64

Unique manufacturers: 476


## 7. Export Results

Export your query results to CSV for further analysis in Excel, R, or other tools.

In [8]:
# Export to CSV
output_file = f'{device_name}_events_{YEAR}.csv'
results.to_csv(output_file, index=False)

print(f"✓ Exported {len(results):,} results to: {output_file}")

✓ Exported 3,837 results to: catheter_events_2005.csv


## 8. Try a Different Device

Now try searching for a different device type. Some examples to try:
- `pacemaker`
- `insulin pump`
- `stent`
- `hip prosthesis`
- `glucose monitor`
- `defibrillator`

Modify the `new_device` variable below and run the cell!

In [9]:
# Try a different device (modify this!)
new_device = 'pacemaker'  # <-- Change this to any device name

sql = """
    SELECT *
    FROM device
    WHERE GENERIC_NAME LIKE :device
       OR BRAND_NAME LIKE :device
"""

new_results = db.query(sql, params={'device': f'%{new_device}%'})
print(f"Found {len(new_results):,} reports for '{new_device}'")

if len(new_results) > 0:
    print(f"\nSample results:")
    display(new_results[available_cols].head())
else:
    print(f"\nNo results found for '{new_device}' in {YEAR}.")
    print("Try a different device name or a more recent year (see notebook 02).")

Found 3,357 reports for 'pacemaker'

Sample results:


,MDR_REPORT_KEY,DATE_RECEIVED,GENERIC_NAME,BRAND_NAME,MODEL_NUMBER
0,564533,2005-01-04 00:00:00,DEFIBRILLATOR/PACEMAKER,M SERIES,MSERIES BI-PHASIC
1,564536,2005-01-04 00:00:00,DEFIBRILLATOR/PACEMAKER,M SERIES,M SERIES BI-PHASIC
2,564542,2005-01-04 00:00:00,DEFIBRILLATOR/PACEMAKER,M SERIES,MSERIES BI-PHASIC
3,564545,2005-01-04 00:00:00,DEFIBRILLATOR/PACEMAKER,M SERIES,MSERIES BI-PHASIC
4,564550,2005-01-04 00:00:00,DEFIBRILLATOR/PACEMAKER,M SERIES,M SERIES BI-PHASIC


## 9. Cleanup

Always close the database connection when done to free up resources.

In [10]:
db.close()

print("✓ Analysis complete!")
print(f"\nDatabase saved to: {DB_PATH}")
print(f"Downloaded data cached in: {DATA_DIR}")
print(f"\nRe-running this notebook will be much faster (data is cached).")

✓ Analysis complete!

Database saved to: quickstart_demo.db
Downloaded data cached in: ./maude_data

Re-running this notebook will be much faster (data is cached).


## Next Steps

Congratulations! You've completed the quick start tutorial. Here's what to explore next:

### For Local Use:
1. **`02_getting_started.ipynb`** - Complete setup guide with multiple years and helper methods
2. **`03_trend_analysis_visualization.ipynb`** - Analyze trends over time with visualizations
3. **`08_helper_methods_reference.ipynb`** - Learn all available helper methods

### For Advanced Analysis:
4. **`04_advanced_querying.ipynb`** - Complex SQL queries and filters
5. **`05_manufacturer_comparison.ipynb`** - Compare safety profiles across manufacturers
6. **`06_signal_detection.ipynb`** - Identify unusual patterns in adverse events
7. **`07_reproducible_research_workflow.ipynb`** - Best practices for publication

### Documentation:
- **`../docs/getting_started.md`** - Detailed installation and setup
- **`../docs/api_reference.md`** - Complete API documentation
- **`../docs/maude_overview.md`** - Understanding the MAUDE database structure
- **`../docs/research_guide.md`** - Best practices for medical device research

### Questions or Issues?
- Check the main README at `../README.md`
- Review troubleshooting guide at `../docs/troubleshooting.md`

---

**Limitations of this quick demo:**
- Uses 2005 data (not recent)
- Device table only (no event types, dates, or patient info)
- Direct SQL queries (no helper methods)

**For full functionality**, see notebooks 02-08 which use the master table and complete API!